In [1]:
from datasets import load_dataset,DatasetDict
from transformers import AutoTokenizer,AutoModelForSequenceClassification, DataCollatorWithPadding

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
raw_datasets = load_dataset("khondoker/SentNoB")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/khondoker--SentNoB-1d4e2e75659b8316/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:154: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_kwargs)
/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:154: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_kwargs)
/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:154: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_

  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['Data', 'Label'],
        num_rows: 15728
    })
})

In [4]:
train_test_valid = raw_datasets['train'].train_test_split(test_size = 0.2)


In [5]:
train_test_valid

DatasetDict({
    train: Dataset({
        features: ['Data', 'Label'],
        num_rows: 12582
    })
    test: Dataset({
        features: ['Data', 'Label'],
        num_rows: 3146
    })
})

In [6]:
test_valid = train_test_valid['test'].train_test_split(test_size=0.5)

In [7]:
test_valid

DatasetDict({
    train: Dataset({
        features: ['Data', 'Label'],
        num_rows: 1573
    })
    test: Dataset({
        features: ['Data', 'Label'],
        num_rows: 1573
    })
})

In [8]:
raw_datasets = DatasetDict({
    'train': train_test_valid['train'],
    'valid':test_valid['train'],
    'test':test_valid['test']
})

In [9]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['Data', 'Label'],
        num_rows: 12582
    })
    valid: Dataset({
        features: ['Data', 'Label'],
        num_rows: 1573
    })
    test: Dataset({
        features: ['Data', 'Label'],
        num_rows: 1573
    })
})

In [10]:
raw_datasets = raw_datasets.rename_column("Label","labels")
raw_datasets = raw_datasets.rename_column("Data","text")
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 12582
    })
    valid: Dataset({
        features: ['text', 'labels'],
        num_rows: 1573
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 1573
    })
})

In [11]:
#raw_datasets = raw_datasets["train"].to_pandas()

In [12]:
checkpoint = 'csebuetnlp/banglabert'

In [13]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels = 3)

Some weights of the model checkpoint at csebuetnlp/banglabert were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at csebuetnlp/banglabert and are newly initialized: ['classifier.dense.weight', 'classifie

In [14]:
def tokenize_fn(examples):
    return tokenizer(examples['text'], truncation = 'longest_first')

In [15]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 12582
    })
    valid: Dataset({
        features: ['text', 'labels'],
        num_rows: 1573
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 1573
    })
})

In [16]:
tokenized_datasets = raw_datasets.map(tokenize_fn, batched = True)

  0%|          | 0/13 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [17]:
tokenized_datasets


DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 12582
    })
    valid: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1573
    })
    test: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1573
    })
})

In [18]:
data_collator = DataCollatorWithPadding(tokenizer)

In [19]:
from transformers import TrainingArguments, Trainer

In [20]:
pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [21]:
import evaluate

In [22]:
def compute_metrics(eval_preds):
    metric = evaluate.combine(["accuracy"])
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [23]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch", per_device_train_batch_size = 16, per_device_eval_batch_size = 16, num_train_epochs = 5, learning_rate = 2e-5, weight_decay = 0.1)

In [24]:
trainer = Trainer(model, training_args, train_dataset = tokenized_datasets['train'], eval_dataset = tokenized_datasets['valid'], data_collator = data_collator, tokenizer = tokenizer)

In [25]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 trainer.train()                                                                              │
│   2                                                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer.py:1645 in train                    │
│                                                                                                  │
│   1642 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1643 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1644 │   │   )                                                                                 │
│ ❱ 1645 │   │   return inner_training_loop(                                                       │
│   1646 │   │   │   args=args,                                                                    │
│   1647 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1648 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer.py:1862 in _inner_training_loop     │
│                                                                                                  │
│   1859 │   │   self._globalstep_last_logged = self.state.global_step                             │
│   1860 │   │   model.zero_grad()                                                                 │
│   1861 │   │                                                                                     │
│ ❱ 1862 │   │   self.control = self.callback_handler.on_train_begin(args, self.state, self.contr  │
│   1863 │   │                                                                                     │
│   1864 │   │   # Skip the first epochs_trained epochs to get the random state of the dataloader  │
│   1865 │   │   if not args.ignore_data_skip:                                                     │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer_callback.py:353 in on_train_begin   │
│                                                                                                  │
│   350 │                                                                                          │
│   351 │   def on_train_begin(self, args: TrainingArguments, state: TrainerState, control: Trai   │
│   352 │   │   control.should_training_stop = False                                               │
│ ❱ 353 │   │   return self.call_event("on_train_begin", args, state, control)                     │
│   354 │                                                                                          │
│   355 │   def on_train_end(self, args: TrainingArguments, state: TrainerState, control: Traine   │
│   356 │   │   return self.call_event("on_train_end", args, state, control)                       │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer_callback.py:397 in call_event       │
│                                                                                                  │
│   394 │                                                                                          │
│   395 │   def call_event(self, event, args, state, control,

In [ ]:
predictions = trainer.predict(tokenized_datasets["valid"])
print(predictions.predictions.shape, predictions.label_ids.shape)

In [ ]:
from sklearn.metrics import classification_report
preds = np.argmax(predictions.predictions, axis=-1)
print(classification_report(y_true=predictions.label_ids, y_pred=preds))

import numpy as np
metric = evaluate.combine(["accuracy","f1"])

preds = np.argmax(predictions.predictions, axis=-1)
scores = metric.compute(predictions=preds, references=predictions.label_ids, average="weighted")

print("scores:", scores)

In [ ]:
predictions = trainer.predict(tokenized_datasets["test"])
print(predictions.predictions.shape, predictions.label_ids.shape)

In [ ]:
from sklearn.metrics import classification_report
preds = np.argmax(predictions.predictions, axis=-1)
print(classification_report(y_true=predictions.label_ids, y_pred=preds))